In [1]:
from brickmapper import Mapper
from rdflib import BRICK
import urllib.request
import csv
import numpy as np



In [2]:
# read haystack proto definitions off their website

defns = []
request = urllib.request.Request("https://project-haystack.org/download/protos.csv", headers={'User-Agent': "Python script"})
response = urllib.request.urlopen(request)
rdr = csv.DictReader(response.read().decode('utf8').splitlines())
for row in rdr:
    # save each proto (e.g. 'air temp sensor point') as a dictionary
    defns.append({'name': row['proto']})

defns

[{'name': 'air co2 concentration sensor point'},
 {'name': 'air co2 concentration sp point'},
 {'name': 'air dewPoint sensor point'},
 {'name': 'air dewPoint sp point'},
 {'name': 'air enthalpy sensor point'},
 {'name': 'air enthalpy sp point'},
 {'name': 'air flow sensor point'},
 {'name': 'air flow sp point'},
 {'name': 'air humidity sensor point'},
 {'name': 'air humidity sp point'},
 {'name': 'air pressure sensor point'},
 {'name': 'air pressure sp point'},
 {'name': 'air temp sensor point'},
 {'name': 'air temp sp point'},
 {'name': 'air velocity sensor point'},
 {'name': 'air velocity sp point'},
 {'name': 'alarm sensor point'},
 {'name': 'avg ac elec current angle sensor point'},
 {'name': 'avg ac elec current imbalance sensor point'},
 {'name': 'avg ac elec current magnitude sensor point'},
 {'name': 'avg ac elec current thd sensor point'},
 {'name': 'avg ac elec pf sensor point'},
 {'name': 'avg ac elec volt angle sensor point'},
 {'name': 'avg ac elec volt imbalance sensor po

In [3]:
# create a new Mapper object which knows about the definitions. This will
# access OpenAI to compute embeddings if necessary
m = Mapper(defns, "haystack_proto.index")

# define which set of Brick classes we want to match against
# all points which aren't parameters (these have similar enough names it can confuse the mapping)
points = m.get_brick_classes(BRICK.Point, [BRICK.Parameter])
# all equipment in Brick
equips = m.get_brick_classes(BRICK.Equipment)
# concatenate these two lists of Brick classes
bcs = points + equips

bcs

2024-12-19 15:40:23.196 | INFO     | brickmapper:populate_external_embeddings:66 - Restored haystack_proto.index


[BrickClassDefinition(class_=rdflib.term.URIRef('https://brickschema.org/schema/Brick#Water_Differential_Temperature_Setpoint'), label='Water Differential Temperature Setpoint', definition='Sets the target differential temperature between the start and end of a heat transfer cycle in a water circuit', definitions=[]),
 BrickClassDefinition(class_=rdflib.term.URIRef('https://brickschema.org/schema/Brick#Temperature_Differential_Reset_Setpoint'), label='Temperature Differential Reset Setpoint', definition=None, definitions=[]),
 BrickClassDefinition(class_=rdflib.term.URIRef('https://brickschema.org/schema/Brick#Differential_Pressure_Sensor'), label='Differential Pressure Sensor', definition='Measures the difference between two applied pressures', definitions=[]),
 BrickClassDefinition(class_=rdflib.term.URIRef('https://brickschema.org/schema/Brick#Heat_Exchanger_Leaving_Water_Temperature_Sensor'), label='Heat Exchanger Leaving Water Temperature Sensor', definition=None, definitions=[]),

In [4]:
# compute a 'stable' mapping (1:1 mapping)
stable = m.get_mapping(bcs, allow_collisions=False)
# compute an 'unstable' mapping that allows 1:n and n:1 matches
unstable = m.get_mapping(bcs, allow_collisions=True)

# loop through and print the stable/unstable matches
for k,v in unstable.items():
    print(f"{k}:\n\tunstable: {v}\n\tstable: {stable.get(k)}")

100%|████████████████████████████████████████████████████████████████████████████████████| 1121/1121 [00:00<00:00, 3263.27it/s]

chilled water delta temp sensor point:
	unstable: (rdflib.term.URIRef('https://brickschema.org/schema/Brick#Water_Differential_Temperature_Setpoint'), 0.19118059)
	stable: https://brickschema.org/schema/Brick#Chilled_Water_Differential_Temperature_Sensor
temp sensor point:
	unstable: (rdflib.term.URIRef('https://brickschema.org/schema/Brick#Temperature_Deadband_Setpoint'), 0.22695565)
	stable: https://brickschema.org/schema/Brick#Core_Temperature_Sensor
water pressure sensor point:
	unstable: (rdflib.term.URIRef('https://brickschema.org/schema/Brick#Pressure_Status'), 0.21616995)
	stable: https://brickschema.org/schema/Brick#Hot_Water_Differential_Pressure_Sensor
chilled water leaving temp sensor point:
	unstable: (rdflib.term.URIRef('https://brickschema.org/schema/Brick#Leaving_Medium_Temperature_Hot_Water_Temperature_Load_Shed_Status'), 0.20692575)
	stable: https://brickschema.org/schema/Brick#Leaving_Chilled_Water_Temperature_Sensor
weather air feelsLike sensor point:
	unstable: (rd

In [5]:
m.external_to_brick_mapping

{rdflib.term.URIRef('https://brickschema.org/schema/Brick#Water_Differential_Temperature_Setpoint'): [('chilled water delta temp sensor point',
   0.19118059),
  ('water temp sensor point', 0.19473952),
  ('chilled water header temp sensor point', 0.19523096),
  ('chilled water delta temp point', 0.19701356),
  ('chilled water entering temp sensor point', 0.19821191),
  ('chilled water delta flow point', 0.19825143),
  ('chilled water leaving temp sensor point', 0.20141846),
  ('hot water delta temp sensor point', 0.20636845),
  ('hot water header temp sensor point', 0.20742923),
  ('chilled water bypass temp sensor point', 0.20902711),
  ('chilled water delta flow sensor point', 0.20906842),
  ('temp sensor point', 0.20934886),
  ('hot water entering temp sensor point', 0.20998836),
  ('chilled water entering temp sp point', 0.21088707),
  ('condenser water entering temp sensor point', 0.21105224),
  ('condenser water delta temp sensor point', 0.21210098),
  ('chilled water header poi